SQLAlchemy is python package which enables working with SQL in Pythonic way. The biggest advantage of SQLAlchemy is that you don't need to deal with different dialects of SQL (Postgres, MySQL, Oracle etc.) but rather with Python objects.

**ORM** - Object Relation Mapper is highest level of sql alchemy.
In shortcut it's the process of assiociating object oriented classes with database tables.

### IMPORTS

In [1]:
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import or_
import pandas as pd

### URL CREATION

In [2]:
SQL_DIALECT = 'postgresql'
DRIVER = 'psycopg2'
USER = 'example'
PASSWORD = 'example'
SERVER = 'postgres'
HOST = 'localhost'
DATABASE = 'example'
PORT = '5432'
CONNECTION_URL = f'{SQL_DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}'

Now it's time to create the engine - it's created once per session and it enables to process all transacitons.

In [3]:
engine = create_engine(
    CONNECTION_URL
)

Let's create our first table: Orders. In order to do that i'm going to use declaritive_base and store it in variable **Base** (table Orders will inherit from it ). We will come back to **Base** a bit later.

In [4]:
Base = declarative_base()

class Order(Base):

    __tablename__ = "orders"

    id = Column(Integer, primary_key=True)
    price = Column(Float)
    amount = Column(Integer)
    product_id = Column(Integer)
    
    def __repr__(self):
        return f"""<Order(price='{self.price}', amount='{self.amount}', product_id='{self.product_id}')>"""

let's create first order and store it in variable, we can do so as follows:

In [5]:
order = Order(
    price=32,
    amount=2,
    product_id = 3
)

At this stage there's nothing in our database if we will look into it:

order is just Python object

In [6]:
print(order)

<Order(price='32', amount='2', product_id='3')>


attributes that weren't set are displayed as None

In [7]:
print(order.id)

None


So let's create table in database now. If we want to do so we need to use our **Base**.

Now we can use context manager (there are beyond scope of this article but for sake of simplicity let's think about context managers as about object which cares about opening/closing the connection with database for you) with engine object - we need to use method begin on engine object in order to commit changes in the database:

In [8]:
with engine.begin() as conn:
    Base.metadata.drop_all(conn)
    Base.metadata.create_all(conn)

But how add the user to the table? in order to do so we will need **Session object**. This object will use **engine** created before and take handle of connecting, commiting and releasing connections to this engine. 

In [9]:
Session:sessionmaker = sessionmaker(bind=engine)
session:sessionmaker = Session() 

New objects can be placed into session using .add method. So let's add our order object created before:

In [10]:
session.add(order)

Note: Nothing happend to the database yet, but we can look into pending objects in our session using .new method

In [11]:
session.new

IdentitySet([<Order(price='32', amount='2', product_id='3')>])

In order to commit we simply use commit method on session object.

In [12]:
session.commit()

After commiting our object order also changes, now .id attribute is not None anymore

In [13]:
print(order.id)

1


It's 1 as excepted.

Multiple objects can be added to the session - in order to do so we will need use method .add_all() as follows:

In [14]:
from typing import List

orders_to_add:List[Order] = [
    Order(
        price = 10,
        amount = 10,
        product_id = 1
    ),

    Order(
        price=15,
        amount=17,
        product_id=2
    )
    ]


session.add_all(orders_to_add)

Let's check

In [15]:
session.new

IdentitySet([<Order(price='10', amount='10', product_id='1')>, <Order(price='15', amount='17', product_id='2')>])

And now commit.

In [16]:
session.commit()

It's also worth to mention that after a commit session invalidates all data, so when we will check session.new it will be empty right now

In [17]:
session.new

IdentitySet([])

How to query our data? Again session object will be useful:

## Select

In [18]:
res = session.query(Order).all()

for r in res:
    print(r)

<Order(price='32.0', amount='2', product_id='3')>
<Order(price='10.0', amount='10', product_id='1')>
<Order(price='15.0', amount='17', product_id='2')>


## Filter

#### simple filter on price

In [19]:
res = (
    session.query(Order).filter(
    Order.price > 1
    ).all()
)

for r in res:
    print(r)

<Order(price='32.0', amount='2', product_id='3')>
<Order(price='10.0', amount='10', product_id='1')>
<Order(price='15.0', amount='17', product_id='2')>


#### Filter multiple columns (works like and)

In [20]:
res = (
    session.query(Order).filter(
    Order.price > 3 ,
    Order.amount > 2
    ).all()
)

for r in res:
    print(r)

<Order(price='10.0', amount='10', product_id='1')>
<Order(price='15.0', amount='17', product_id='2')>


#### filter multiple columns with or

In [21]:
res = (
    session.query(Order).filter(
    or_(
    Order.price > 3,
    Order.amount > 2)
    ).all()
)

for r in res:
    print(r)

<Order(price='32.0', amount='2', product_id='3')>
<Order(price='10.0', amount='10', product_id='1')>
<Order(price='15.0', amount='17', product_id='2')>


#### In:

In [22]:
res = (
    session.query(Order).filter(
    Order.product_id.in_((1, 2)
    )
    ).all()
)


for r in res:
    print(r)

<Order(price='10.0', amount='10', product_id='1')>
<Order(price='15.0', amount='17', product_id='2')>


### Order by

In [23]:
res = (
    session.query(Order).order_by(Order.product_id)
    ).all()


for r in res:
    print(r)

<Order(price='10.0', amount='10', product_id='1')>
<Order(price='15.0', amount='17', product_id='2')>
<Order(price='32.0', amount='2', product_id='3')>


### To pandas dataframe

In [24]:
df:pd.DataFrame = pd.read_sql(
    sql=session.query(Order).statement,
    con=session.bind
)

In [25]:
df

,id,price,amount,product_id
0,1,32.0,2,3
1,2,10.0,10,1
2,3,15.0,17,2
